In [1]:
import torch.utils
import torch.utils.data
import torch.utils.data.dataset
import torchvision
import numpy as np
from torch.utils.data import TensorDataset
import torch
from utils.inputdata import InputData
import random
from utils.runner import BaseRunner
import torch.utils
import torch.utils.data
from models.som import SOM
import models.som
from torch.utils.data import TensorDataset
import torch
import matplotlib.pyplot as plt
from utils.inputdata import InputData
import argparse
import wandb
import random
from abc import ABC, abstractmethod
from typing import Any
import importlib
importlib.reload(models.som)
from models.som import SOM
from models.stm import STM
import math

In [14]:
for i in range(40):
	print(math.exp(-i*0.005))

1.0
0.9950124791926823
0.9900498337491681
0.9851119396030626
0.9801986733067553
0.9753099120283326
0.9704455335485082
0.9656054162575665
0.9607894391523232
0.9559974818331
0.951229424500714
0.9464851479534838
0.9417645335842487
0.9370674633774034
0.9323938199059483
0.9277434863285529
0.9231163463866358
0.9185122844014574
0.9139311852712282
0.9093729344682314
0.9048374180359595
0.9003245225862656
0.8958341352965282
0.8913661439068313
0.8869204367171575
0.8824969025845955
0.8780954309205613
0.8737159116880344
0.8693582353988059
0.8650222931107413
0.8607079764250578
0.8564151774836135
0.8521437889662113
0.8478937040879159
0.8436648165963837
0.8394570207692074
0.835270211411272
0.8311042838521256
0.8269591339433623
0.8228346580560184
